In [ ]:
!pip install -q -U google-genai

In [ ]:
'''SERP API INTEGRATION'''
import requests
import json # Necessary for json.dumps()
from google.genai.types import FunctionDeclaration, Type, Schema
from google.colab import userdata

# NOTE: Replace 'YOUR_SERPAPI_KEY' with your actual key from the service
SERPAPI_API_KEY = 'YOUR_SERPAPI_KEY'

def serp_job_search_tool(query: str) -> str:
    """
    Uses an external SERP API to fetch structured job data, including application links.
    Returns the result as a JSON string for the LLM to process.
    """
    if not globals().get('SERPAPI_API_KEY') or globals()['SERPAPI_API_KEY'] == 'PLACEHOLDER':
        return json.dumps({"error": "SERPAPI_API_KEY is not configured. Live search failed."})

    # Actual SERP API Call (Example using SerpApi)
    params = {
      "api_key": SERPAPI_API_KEY,
      "engine": "google_jobs",
      "q": query,
      "hl": "en",
      "gl": "in",
    }

    try:
        # Make the actual API request
        response = requests.get("https://serpapi.com/search", params=params)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

        data = response.json()

        # We only care about the structured job results section
        jobs_results = data.get('jobs_results', [])

        if not jobs_results:
            return json.dumps({"error": "No jobs found for the query.", "raw_response": data})

        # Return the clean, structured list of job results as a JSON string
        return json.dumps({"jobs_results": jobs_results})

    except requests.exceptions.RequestException as e:
        return json.dumps({"error": f"API Request failed: {e}"})
    except Exception as e:
        return json.dumps({"error": f"An unexpected error occurred: {e}"})


# 2. Function Declaration for the SERP tool
SERP_TOOL_DECLARATION = FunctionDeclaration(
    name='serp_job_search_tool',
    description='Fetches structured job postings and direct application links for a given query using a powerful external API.',
    parameters=Schema(
        type=Type.OBJECT,
        properties={
            'query': Schema(type=Type.STRING, description='The job title and location to search for.'),
        },
        required=['query']
    )
)

In [ ]:
import os
import json
import requests
from google.colab import userdata

# --- CRITICAL IMPORTS FOR GEMINI SDK TYPES ---
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    Type,
    Schema,
    Tool,
    GenerateContentConfig,
    Part,
    Content
)
from google.genai import types

# --- SETUP: Configuration and API Key Loading ---
print("--- 1. Setting up the environment ---")

try:
    print("Installing/Updating necessary libraries...")
    !pip install -q -U google-genai requests
except Exception as e:
    print(f"Error during package installation: {e}")

# Initialize placeholders
API_KEY = None
SERPAPI_API_KEY = None
client = None

# Load API Keys from Colab Secrets
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in Colab Secrets.")

    # FIXED: Use the correct secret name
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')  # Changed from YOUR_SERPAPI_KEY
    if not SERPAPI_API_KEY:
        raise ValueError("SERPAPI_API_KEY not found in Colab Secrets.")

    client = genai.Client(api_key=API_KEY)
    AGENT_MODEL = 'gemini-2.5-flash'
    print("✅ Gemini API client and SERPAPI_API_KEY configured successfully.")
    print(f"   SERPAPI_API_KEY loaded: {SERPAPI_API_KEY[:10]}..." if SERPAPI_API_KEY else "   No key loaded")

except Exception as e:
    print(f"❌ ERROR: Configuration failed. Details: {e}")
    AGENT_MODEL = 'gemini-2.5-flash'


# --- CUSTOM TOOL DEFINITIONS ---

def serp_job_search_tool(query: str) -> str:
    """
    Uses the live SerpApi to fetch structured Google Jobs data.
    Enhanced with better error handling and debugging.
    """
    print(f"\n🔍 SERP Tool Called with query: '{query}'")

    if not SERPAPI_API_KEY or SERPAPI_API_KEY == 'PLACEHOLDER':
        error_msg = "SERPAPI_API_KEY is missing or invalid."
        print(f"❌ {error_msg}")
        return json.dumps({"error": error_msg, "jobs_results": []})

    # IMPROVED: More flexible search parameters
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google_jobs",
        "q": query,
        "hl": "en",
        "gl": "in",
        # REMOVED: "chips": "date_posted:month" - This might be too restrictive
    }

    try:
        print(f"   Making API request to SerpAPI...")
        response = requests.get("https://serpapi.com/search", params=params, timeout=10)

        # Debug: Print status code
        print(f"   Response status: {response.status_code}")

        response.raise_for_status()
        data = response.json()

        # Debug: Print API response structure
        print(f"   API Response keys: {list(data.keys())}")

        # Check for API errors
        if "error" in data:
            error_msg = f"SerpAPI Error: {data['error']}"
            print(f"❌ {error_msg}")
            return json.dumps({"error": error_msg, "jobs_results": []})

        jobs_results = data.get('jobs_results', [])

        print(f"   Jobs found: {len(jobs_results)}")

        if not jobs_results:
            # IMPROVED: Return more diagnostic information
            return json.dumps({
                "jobs_results": [],
                "error": "No jobs found for the query.",
                "debug_info": {
                    "query_used": query,
                    "search_metadata": data.get("search_metadata", {}),
                    "suggestion": "Try a broader search term like 'Software Developer India' or 'Java Developer'"
                }
            })

        # Success: Return structured results
        print(f"✅ Successfully retrieved {len(jobs_results)} jobs")
        return json.dumps({"jobs_results": jobs_results[:5]})  # Limit to top 5

    except requests.exceptions.Timeout:
        error_msg = "API request timed out. Please try again."
        print(f"❌ {error_msg}")
        return json.dumps({"error": error_msg, "jobs_results": []})

    except requests.exceptions.RequestException as e:
        error_msg = f"API Request failed: {str(e)}"
        print(f"❌ {error_msg}")
        return json.dumps({"error": error_msg, "jobs_results": []})

    except Exception as e:
        error_msg = f"Unexpected error: {str(e)}"
        print(f"❌ {error_msg}")
        return json.dumps({"error": error_msg, "jobs_results": []})


# Function Declaration for the SERP tool
SERP_TOOL_DECLARATION = FunctionDeclaration(
    name='serp_job_search_tool',
    description='Fetches structured job postings including title, company, link, and description for a given query.',
    parameters=Schema(
        type=Type.OBJECT,
        properties={'query': Schema(type=Type.STRING, description='The job title and location to search for.')},
        required=['query']
    )
)


def log_job_to_db(job_title: str, company: str, status: str) -> str:
    """Logs a job posting to simulated database."""
    log_entry = {"title": job_title, "company": company, "status": status}
    print(f"📝 [JobDatabaseTool]: Logging job: {json.dumps(log_entry)}")
    return f"Job '{job_title}' successfully logged with status '{status}'."


LOG_TOOL_DECLARATION = FunctionDeclaration(
    name='log_job_to_db',
    description='Logs a job posting and its status to a simulated persistent database.',
    parameters=Schema(
        type=Type.OBJECT,
        properties={
            'job_title': Schema(type=Type.STRING, description='The exact title of the job posting.'),
            'company': Schema(type=Type.STRING, description='The name of the company offering the job.'),
            'status': Schema(type=Type.STRING, description='The current review status.')
        },
        required=['job_title', 'company', 'status']
    )
)


# --- AGENT DEFINITIONS ---

def search_agent(query: str, client: genai.Client, model: str) -> str:
    """Search Agent - Uses SERP tool to find structured job postings."""
    if client is None:
        return "No job data found. Gemini client not initialized."

    print(f"\n⚙️ Search Agent received query: '{query}'")

    system_instruction = (
        "You are the expert Search Agent. You **must** use the `serp_job_search_tool` "
        "to find relevant job postings. When you receive the structured JSON data, your **PRIMARY** goal is to extract " "the Title, Company, Location, Description, and the **FULL APPLICATION URL**. "
        "Format the data into a clean, numbered list, ensuring the **Application Link** is clearly labeled and included for every job. "
        "This link must be preserved exactly as it appears in the JSON (usually under 'apply_link' or 'related_links')."
    )

    tools = [Tool(function_declarations=[SERP_TOOL_DECLARATION])]

    prompt = f"Find job postings for: {query}"

    # First call: Request function call
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            tools=tools,
            tool_config=types.ToolConfig(
                function_calling_config=types.FunctionCallingConfig(mode='ANY')
            )
        )
    )

    # Tool Execution
    if response.function_calls:
        print("    [Function Call Detected]: Executing serp_job_search_tool...")

        call = response.function_calls[0]
        job_data_json = serp_job_search_tool(**dict(call.args))

        # Parse to check for errors
        job_data = json.loads(job_data_json)

        # IMPROVED: Better error handling
        if job_data.get("error"):
            error_message = job_data["error"]
            debug_info = job_data.get("debug_info", {})
            suggestion = debug_info.get("suggestion", "Try different search terms")

            return f"⚠️ Search failed: {error_message}\n\n💡 Suggestion: {suggestion}"

        # Second call: Get formatted response
        response = client.models.generate_content(
            model=model,
            contents=[
                Content(role='user', parts=[Part(text=prompt)]),
                Content(role='model', parts=[Part(function_call=call)]),
                Content(role='function', parts=[Part(function_response=types.FunctionResponse(
                    name='serp_job_search_tool',
                    response={'job_data': job_data_json}
                ))])
            ],
            config=GenerateContentConfig(system_instruction=system_instruction)
        )

        print("✅ Search Agent finished.")
        return response.text
    else:
        print("⚠️ Warning: Search Agent did not call the SERP tool.")
        return "No job data found. Model failed to execute tool call."


def analysis_agent(job_list_text: str, user_profile: str, client: genai.Client, model: str) -> str:
    """Analysis Agent - Reviews and scores job matches."""
    print("\n⚙️ Analysis Agent started review...")

    system_instruction = (
        "You are the Analysis Agent. Review the job postings against the user profile, "
        "log each job using `log_job_to_db`, then provide detailed analysis with match scores (1-10)."
    )

    tools = [LOG_TOOL_DECLARATION]

    prompt = (
        f"Analyze these jobs against this profile: {user_profile}\n\n"
        f"For each job: 1) Log it with log_job_to_db, 2) Provide analysis and score.\n\n"
        f"Jobs:\n{job_list_text}"
    )

    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=GenerateContentConfig(system_instruction=system_instruction, tools=tools)
    )

    # Handle logging tool calls
    function_responses = []
    for part in response.candidates[0].content.parts:
        if part.function_call and part.function_call.name == 'log_job_to_db':
            call = part.function_call
            log_result = log_job_to_db(**dict(call.args))
            function_responses.append(Part(function_response=types.FunctionResponse(
                name='log_job_to_db',
                response={'result': log_result}
            )))

    if function_responses:
        response = client.models.generate_content(
            model=model,
            contents=[
                Content(role='user', parts=[Part(text=prompt)]),
                response.candidates[0].content,
                Content(role='function', parts=function_responses)
            ],
            config=GenerateContentConfig(system_instruction=system_instruction)
        )

    print("✅ Analysis Agent finished.")
    return response.text


def recruiter_agent(prompt: str, user_profile: str, client: genai.Client, model: str) -> str:
    """Recruiter Agent - Orchestrates the job search process."""
    print("\n" + "="*50)
    print("🎯 RECRUITER AGENT STARTING")
    print("="*50)

    search_query = prompt
    job_list_text = search_agent(search_query, client, model)

    # Check for search failure
    if "Search failed" in job_list_text or "No job data found" in job_list_text:
        return f"**Job Search Failed**\n\n{job_list_text}"

    final_analysis_text = analysis_agent(job_list_text, user_profile, client, model)

    # Synthesis
    system_instruction = (
        "You are the Recruiter Agent. Synthesize the analysis into a clear job recommendation report. "
        "Present the top matches with scores and application links."
    )

    synthesis_prompt = (
        f"Create a final job recommendation report from this analysis. "
        f"Highlight the top 2 best matches and **ensure the full application URL is clearly displayed** "
        f"for each highlighted job, using a bold 'Application Link:' label. User request: '{prompt}'\n\nAnalysis:\n{final_analysis_text}"
    )

    final_response = client.models.generate_content(
        model=model,
        contents=synthesis_prompt,
        config=GenerateContentConfig(system_instruction=system_instruction)
    )

    print("="*50)
    print("✅ RECRUITER AGENT FINISHED")
    print("="*50)
    return final_response.text


# --- MAIN EXECUTION ---

USER_PROFILE = """
I am a Software Engineer
Skills: Java, Javascript, React, Spring-boot, Agile.
"""

USER_JOB_REQUEST = "Software Engineer jobs in India"

print("\n\n" + "#"*50)
print("🔥 AI AGENTS RECRUITER SYSTEM")
print("#"*50 + "\n")

if client:
    final_report = recruiter_agent(
        prompt=USER_JOB_REQUEST,
        user_profile=USER_PROFILE,
        client=client,
        model=AGENT_MODEL
    )
    print("\n\n" + "="*50)
    print("📊 FINAL REPORT")
    print("="*50)
    print(final_report)
else:
    print("❌ Cannot run: Gemini client not initialized. Check your API keys.")

--- 1. Setting up the environment ---
Installing/Updating necessary libraries...
✅ Gemini API client and SERPAPI_API_KEY configured successfully.
   SERPAPI_API_KEY loaded: ad218a15e4...


##################################################
🔥 AI AGENTS RECRUITER SYSTEM
##################################################


🎯 RECRUITER AGENT STARTING

⚙️ Search Agent received query: 'Software Engineer jobs in India'
    [Function Call Detected]: Executing serp_job_search_tool...

🔍 SERP Tool Called with query: 'Software Engineer jobs in India'
   Making API request to SerpAPI...
   Response status: 200
   API Response keys: ['search_metadata', 'search_parameters', 'filters', 'jobs_results', 'serpapi_pagination']
   Jobs found: 10
✅ Successfully retrieved 10 jobs
✅ Search Agent finished.

⚙️ Analysis Agent started review...
✅ Analysis Agent finished.
✅ RECRUITER AGENT FINISHED


📊 FINAL REPORT
Here is your personalized Job Recommendation Report based on the analysis of available "Software E

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load email + app password from Colab/Kaggle secrets
SENDER_EMAIL = userdata.get("MY_EMAIL")
APP_PASSWORD = userdata.get("MY_EMAIL_APP_PASSWORD")

if not SENDER_EMAIL or not APP_PASSWORD:
    print("❌ Email credentials missing in secrets!")
else:
    print("✅ Email system ready.")


✅ Email system ready.


In [ ]:
def send_email(to_email: str, subject: str, body: str):
    """
    Sends an email using Gmail SMTP and App Password.
    """
    try:
        print("\n📨 Sending email...")

        msg = MIMEMultipart()
        msg["From"] = SENDER_EMAIL
        msg["To"] = to_email
        msg["Subject"] = subject

        msg.attach(MIMEText(body, "plain"))

        # Gmail SMTP
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(SENDER_EMAIL, APP_PASSWORD)

        server.sendmail(SENDER_EMAIL, to_email, msg.as_string())
        server.quit()

        print("✅ Email sent successfully!")

    except Exception as e:
        print(f"❌ Email sending failed: {e}")


In [ ]:
if client:
    final_report = recruiter_agent(
        prompt=USER_JOB_REQUEST,
        user_profile=USER_PROFILE,
        client=client,
        model=AGENT_MODEL
    )
    print("\n\n" + "="*50)
    print("📊 FINAL REPORT")
    print("="*50)
    print(final_report)

    # === AUTOMATIC EMAIL DELIVERY ===
    RECIPIENT_EMAIL = userdata.get("MY_EMAIL")  # send to yourself

    print("\n📨 Auto-sending report to your email...")
    send_email(
        to_email=RECIPIENT_EMAIL,
        subject="Your Daily AI Job Report",
        body=final_report
    )
else:
    print("❌ Cannot run: Gemini client not initialized. Check your API keys.")



🎯 RECRUITER AGENT STARTING

⚙️ Search Agent received query: 'Software Engineer jobs in India'
    [Function Call Detected]: Executing serp_job_search_tool...

🔍 SERP Tool Called with query: 'Software Engineer jobs in India'
   Making API request to SerpAPI...
   Response status: 200
   API Response keys: ['search_metadata', 'search_parameters', 'filters', 'jobs_results', 'serpapi_pagination']
   Jobs found: 10
✅ Successfully retrieved 10 jobs
✅ Search Agent finished.

⚙️ Analysis Agent started review...
✅ Analysis Agent finished.
✅ RECRUITER AGENT FINISHED


📊 FINAL REPORT
## Job Recommendation Report for Software Engineer (India)

Based on your profile as a **Software Engineer** with skills in **Java, Javascript, React, Spring-boot, and Agile**, the following analysis provides tailored job recommendations. Your core strengths lie in backend development with Java/Spring-boot and frontend development with React, along with a strong understanding of Agile methodologies.

### Overall Rec

In [ ]:
# !pip install crewai

In [ ]:

# from crewai.scheduler import Scheduler

# USER_PROFILE = """
# I am a Software Engineer
# Skills: Java, Javascript, React, Spring-boot, Agile.
# """

# USER_JOB_REQUEST = "Software Engineer jobs in India for freshers"

# scheduler = Scheduler()

# @scheduler.daily(hour=7, minute=0)
# def run_agent_daily():
#     final_report = recruiter_agent(
#         prompt=USER_JOB_REQUEST,
#         user_profile=USER_PROFILE,
#         client=client,
#         model=AGENT_MODEL
#     )
#     send_email(
#         to_email="your_email@gmail.com",
#         subject="Your Daily Job Report",
#         body=final_report
#     )

# if __name__ == "__main__":
#     scheduler.start()


ModuleNotFoundError: No module named 'crewai.scheduler'